# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [22]:
import azureml.core
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Workspace, ScriptRunConfig, Environment, Experiment

import os
import joblib
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Initializing a Workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-190098
aml-quickstarts-190098
southcentralus
1b944a9b-fdae-4f97-aeb1-b7eea0beac53


## Creating an HyperDrive Experiment

In [3]:
experiment_name = 'hyperdrive-heart-failure'
experiment = Experiment(ws, experiment_name)

## Creating a Compute Cluster

In [4]:
cluster_name = "hd-cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)


print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-03-27T15:15:01.844000+00:00', 'errors': None, 'creationTime': '2022-03-27T09:09:18.107853+00:00', 'modifiedTime': '2022-03-27T09:09:21.692231+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

## Data Wrangling

## Data Gathering

### Overview
In this project, the Heart Failure Prediction dataset from Kaggle is used. The description of the dataset is provided below.

**The dataset has the following features:**  
* age: Age (numeric)
* anaemia: Decrease of red blood cells or hemoglobin (boolean)
* creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
* diabetes: If the patient has diabetes (boolean)
* ejection_fraction: Percentage of blood leaving the heart at each contraction (percentage)
* high_blood_pressure: If the patient has hypertension (boolean)
* platelets: Platelets in the blood (kiloplatelets/mL)
* serum_creatinine: Level of serum creatinine in the blood (mg/dL)
* serum_sodium: Level of serum sodium in the blood (mEq/L)
* sex: Woman or man (binary)

**The task:**  
Developing a ML model to predict death events using 12 clinical features.

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

Following two hyperparameters are selected:  
 
**C:** The regularization strength. Regularization generally refers the concept that there should be a complexity penalty for more extreme parameters. The idea is that just looking at the training data and not paying attention to how extreme one's parameters are leads to overfitting. A high value of C tells the model to give high weight to the training data, and a lower weight to the complexity penalty. A low value tells the model to give more weight to this complexity penalty at the expense of fitting to the training data. Basically, a high C means "Trust this training data a lot", while a low value says "This data may not be fully representative of the real world data, so if it's telling you to make a parameter really large, don't listen to it".

Reference:https://stackoverflow.com/questions/67513075/what-is-c-parameter-in-sklearn-logistic-regression

**max_iter**: The number of iterations.

In [31]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2,
                                        slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1.0, 10.0, 100.0),
        '--max_iter': choice(5, 10, 20, 50, 200)
    }
)


## Creating a training folder if it's not available
if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
env = Environment.get(workspace = ws, name = 'AzureML-Tutorial')
computer_target = ws.compute_targets[cluster_name]

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=env
                      )


hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=src,
                                     max_concurrent_runs=3,
                                     max_total_runs=15,                                     
                                    )


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [32]:
#TODO: Submit your experiment
hdr = experiment.submit(config = hyperdrive_run_config)

#monitoring the experiment
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_630f09c7-0047-4fb3-abda-da0dfef46301
Web View: https://ml.azure.com/runs/HD_630f09c7-0047-4fb3-abda-da0dfef46301?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-190098/workspaces/quick-starts-ws-190098&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-27T15:59:12.104805][API][INFO]Experiment created<END>\n""<START>[2022-03-27T15:59:12.871669][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2022-03-27T15:59:13.484495][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_630f09c7-0047-4fb3-abda-da0dfef46301
Web View: https://ml.azure.com/runs/HD_630f09c7-0047-4fb3-abda-da0dfef46301?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-190098/workspaces/quick-starts-ws-190098&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_630f09c7-0047-4fb3-abda-da0dfef46301',
 'target': 'hd-cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-03-27T15:59:11.862864Z',
 'endTimeUtc': '2022-03-27T16:08:55.871413Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '16076551-a3e0-4fb5-b724-34c842d3d967',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '25',
  'score': '0.7777777777777778',
  'best_child_run_id': 'HD_630f09c7-0047-4fb3-abda-da0dfef46301_4',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg190098.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_630f09c7-0047-4fb3-abda-da0dfef46301/azureml-logs/h

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [35]:
#get the best run
best_run = hdr.get_best_run_by_primary_metric()
print("Best run: ", best_run, sep="\n" )
print("\nBest run metrics: ", best_run.get_metrics()['Accuracy'], sep="\n" )

Best run: 
Run(Experiment: hyperdrive-heart-failure,
Id: HD_630f09c7-0047-4fb3-abda-da0dfef46301_4,
Type: azureml.scriptrun,
Status: Completed)

Best run metrics: 
0.7777777777777778


In [38]:
#TODO: Save the best model
best_model = best_run.register_model(model_name = 'heart-failure-lr', model_path='./outputs/model_lr.joblib')
best_run.download_file('./outputs/model_lr.joblib', 'hdr_model.joblib')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model_lr.joblib in the set of files uploaded to the run: ['outputs/automl_model.pkl', 'outputs/hyper-model.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/model_lr.joblib in the set of files uploaded to the run: ['outputs/automl_model.pkl', 'outputs/hyper-model.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'syste

In [ ]:
# Cleanining up the allocated resources
#compute_cluster.delete()

## Model Deployment

The best ML model is achieved by AutoML. Thus, the model which is trained by the hyperdrive is not deployed.